In [ ]:
# import scripts.interoperabilite as intop
import geopandas as gpd
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, URL
from getpass import getpass
from ast import literal_eval

In [ ]:
psw = getpass("tbadie's password")
# psw = psw.replace('@', '%40')

url_agsdev_agsprodcopy = URL.create(
    "postgresql+psycopg2",
    username = "tbadie",
    password = psw,
    host = "147.100.179.208",
    port = '5438',
    database = "entrepot_20240923",
)
del(psw)

connection = create_engine(url_agsdev_agsprodcopy)

refloc = pd.read_sql("SELECT codeinsee, topiaid FROM reflocation", connection)
refloc = refloc.rename(columns={'codeinsee':'CODGEO_2013'})

In [ ]:
path_csv = '/home/administrateur/Bureau/Datagrosyst/data_entrepot_&_outils/referentiels/geospatial_data/insee/'

mig = pd.read_excel(path_csv+'ref_passage_INSEE.xlsx', dtype='str')
mig = mig[['CODGEO_2024',
           'LIBGEO_2024',
           'CODGEO_2022',
           'CODGEO_2017',
           'CODGEO_2014',
           'CODGEO_2013']]
mig['CODGEO_2024'] = mig['CODGEO_2024'].str.rjust(5, '0')
mig['CODGEO_2022'] = mig['CODGEO_2022'].str.rjust(5, '0')
mig['CODGEO_2017'] = mig['CODGEO_2017'].str.rjust(5, '0')
mig['CODGEO_2013'] = mig['CODGEO_2013'].str.rjust(5, '0')
mig['CODGEO_2014'] = mig['CODGEO_2014'].str.rjust(5, '0')
mig = mig.replace('00000', None)

geo_DG = pd.read_csv(path_csv+'referentiel_geographique_fr_esr_unique.csv', encoding='latin1', dtype='str')
geo_DG = geo_DG[['Municipality_code','Vineyard_basin','Region_old']].rename(columns={'Municipality_code' : 'CODGEO_2022'})
geo_DG['CODGEO_2022'] = geo_DG['CODGEO_2022'].fillna('ZZZZZ')

# latlon = pd.read_csv(path_csv+'lat_long.csv', dtype='str')
# latlon['code_commune_INSEE'] = latlon['code_commune_INSEE'].str.rjust(5, '0')
# latlon['code_departement'] = latlon['code_departement'].str.rjust(2, '0')
# latlon = latlon.loc[~latlon['code_commune_INSEE'].duplicated() & latlon['latitude'].notna()]
# latlon = latlon[latlon.columns.difference(['nom_commune_complet'])].rename(columns={'code_commune_INSEE' : 'CODGEO_2024'})

# zipcode = pd.read_csv(path_csv+'zipcode.csv', encoding='latin1', dtype='str', sep=';')
# zipcode = zipcode[['#Code_commune_INSEE','Code_postal']].rename(columns={'#Code_commune_INSEE' : 'CODGEO_2024', 'Code_postal' : 'codepostal'})
# zipcode = zipcode.loc[~zipcode['CODGEO_2024'].duplicated() & zipcode['codepostal'].notna()]

zipcode = pd.read_csv(path_csv+'zip lat lon geofla.csv', encoding='latin1', dtype='str', sep=';')
zipcode = zipcode[['INSEE_COM','coordonnees_gps','Code_postal','ID_GEOFLA','Z_MOYEN','SUPERFICIE','POPULATION']]
zipcode = zipcode.rename(columns={'INSEE_COM' : 'CODGEO_2014',
                                  'coordonnees_gps' : 'gps',
                                  'Code_postal' : 'codepostal',
                                  'ID_GEOFLA' : 'GEOFLA2015_id',
                                  'Z_MOYEN' : 'altitudemoy',
                                  'SUPERFICIE' : 'area2015',
                                  'POPULATION' : 'pop2015'})
zipcode = zipcode.loc[~zipcode['CODGEO_2014'].duplicated()]

pra = pd.read_excel(path_csv+'pra_vs_codeinsee.xls', dtype='str')
pra = pra[['CODGEO','PRA_Code','PRA_Lib']].rename(columns={'CODGEO' : 'CODGEO_2017'})

comm24 = pd.read_excel(path_csv+'commune_et_plus_2024.xlsx', dtype='str')
comm24 = comm24[comm24.columns.difference(['commune_nom'])].rename(columns={'codeinsee' : 'CODGEO_2024'})

In [ ]:
print(len(mig))
df = mig.merge(refloc, on='CODGEO_2013', how='outer') #+0
print(len(df))
df = df.merge(comm24, on='CODGEO_2024',  how='outer') #+0
print(len(df))
# df = df.merge(latlon, on='CODGEO_2024', how='outer') #+950
# print(len(df))
# df = df.merge(zipcode, on='CODGEO_2024', how='outer') #+90
# print(len(df))
df = df.merge(zipcode, on='CODGEO_2014', how='outer') #+1
print(len(df))
df = df.merge(pra, on='CODGEO_2017', how='outer') #+0
print(len(df))
df = df.merge(geo_DG, on='CODGEO_2022', how='outer') #+0
print(len(df))


In [ ]:
# on enleve les deux lignes entierement NA
df = df.loc[~df.CODGEO_2024.isna()]
df = df.loc[~df.duplicated()]
df['topiaversion'] = '1'
df['topiacreatedate'] = '20250106'
df['timestamp'] = 'xxxx'
df['active'] = '1'
df['refcountry'] = 'fr.inra.agrosyst.api.entities.referential.RefCountry_454036f3-ecf2-472c-9158-b24e60bb24d1'

df = df[['topiaid',
        'topiaversion',
        'CODGEO_2024',
        'timestamp',
        'LIBGEO_2024',
        'PRA_Code', 
        'PRA_Lib',
        'dep_code',
        'codepostal',
        'reg_code',
        'active',
        'gps',
        'refcountry',
        # +
        'altitudemoy', 
        'area2015', 
        'pop2015', 
        'aire_attr_2020', 
        'arr_code',
        'bassin_vie_2022',
        'intercomm_code', 
        'unit_urb_2020', 
        'z_emploi_2020',
        'GEOFLA2015_id',
        'Vineyard_basin', 
        'Region_old',
        'CODGEO_2022',
        'CODGEO_2017', 
        'CODGEO_2014',
        'CODGEO_2013']]

# def alt_moy(x):
#    a = fdf.loc[x.index]
#    f = 0
#    for idx, alt in enumerate(a['altitudemoy']):
#      f = f + (alt * (a[idx]['area2015'] / sum(a['area2015'])))
#    return f

# okdf = df.loc[~(df.CODGEO_2024.duplicated(keep = False) | df.topiaid.duplicated(keep = False))]
# caredf = df.loc[df.topiaid.duplicated(keep = False)]
# fdf = df.loc[df.CODGEO_2024.duplicated(keep = False)].astype

fdf = df.groupby(['CODGEO_2024','LIBGEO_2024'],as_index=False).aggregate({
    'topiaid' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'topiaversion' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'timestamp': lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'PRA_Code' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'PRA_Lib' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'dep_code' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'codepostal' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'reg_code' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'active' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'gps' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'refcountry' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    # +
    'altitudemoy' : lambda x: np.nan if all(x.isna()) else np.ceil(x.astype('Int64').mean()).astype(int), 
    'area2015' : lambda x: x.astype('Int64').sum(), 
    'pop2015' : lambda x: x.astype('Int64').sum(),
    'aire_attr_2020' : lambda x: x.sort_values(na_position='last').iloc[0], 
    'arr_code' : lambda x: x.sort_values(na_position='last').iloc[0],
    'bassin_vie_2022' : lambda x: x.sort_values(na_position='last').iloc[0], 
    'intercomm_code' : lambda x: x.sort_values(na_position='last').iloc[0], 
    'unit_urb_2020' : lambda x: x.sort_values(na_position='last').iloc[0], 
    'z_emploi_2020' : lambda x: x.sort_values(na_position='last').iloc[0],
    'GEOFLA2015_id' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0], 
    'Vineyard_basin' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0], 
    'Region_old' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'CODGEO_2022' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'CODGEO_2017' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'CODGEO_2014' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    'CODGEO_2013' : lambda x: list(set(x.dropna())) if len(set(x.dropna())) > 1 else x.sort_values(na_position='last').iloc[0],
    })

# fdf.to_csv(path_csv + 'refloc_v3.csv', index=False)
# caredf.to_csv(path_csv + 'refloc_duplic.csv', index=False)
# okdf.to_csv(path_csv + 'refloc_ok.csv', index=False)

In [ ]:
fdff = fdf
fdff['temp_checking'] = np.where(fdff['topiaid'].str.contains('\['), 'fusion', 'un_pour_un')
fdff.loc[fdff.topiaid.duplicated(keep = False),'temp_checking'] = 'scission'

check1 = fdff.loc[fdff.PRA_Code.str.contains('\[', na=True), ['CODGEO_2024','LIBGEO_2024','PRA_Code','PRA_Lib','temp_checking']]
check2 = fdff.loc[fdff.gps.str.contains('\[', na=True), ['CODGEO_2024','LIBGEO_2024','gps','temp_checking']]

update = pd.read_csv(path_csv+'refloc_check_v1.csv', dtype='str')
fdff = fdff.merge(update.loc[update.CODGEO_2024.isin(list(check1.CODGEO_2024)),['CODGEO_2024','PRA_Code','PRA_Lib']], on='CODGEO_2024', how='left', suffixes=('', '_new'))
fdff['PRA_Code'] = np.where(pd.notnull(fdff['PRA_Code_new']), fdff['PRA_Code_new'], fdff['PRA_Code'])
fdff['PRA_Lib'] = np.where(pd.notnull(fdff['PRA_Lib_new']), fdff['PRA_Lib_new'], fdff['PRA_Lib'])
fdff.drop('PRA_Code_new', axis=1, inplace=True)
fdff.drop('PRA_Lib_new', axis=1, inplace=True)

fdff = fdff.merge(update.loc[update.CODGEO_2024.isin(list(check2.CODGEO_2024)), ['CODGEO_2024','gps']], on='CODGEO_2024', how='left', suffixes=('', '_new'))
fdff['gps'] = np.where(pd.notnull(fdff['gps_new']), fdff['gps_new'], fdff['gps'])
fdff.drop('gps_new', axis=1, inplace=True)

In [ ]:
fdff.loc[fdff.codepostal.str.contains('\[', na=True), ['CODGEO_2024','LIBGEO_2024','codepostal','temp_checking']]

fdff.loc[fdff.CODGEO_2024 == '13055', 'codepostal'] = '13000'
fdff.loc[fdff.CODGEO_2024 == '18173', 'codepostal'] = '18130'
fdff.loc[fdff.CODGEO_2024 == '51457', 'codepostal'] = '51480'
fdff.loc[fdff.CODGEO_2024 == '60694', 'codepostal'] = '60390'
fdff.loc[fdff.CODGEO_2024 == '64300', 'codepostal'] = '64170'
fdff.loc[fdff.CODGEO_2024 == '69123', 'codepostal'] = '69000'
fdff.loc[fdff.CODGEO_2024 == '69149', 'codepostal'] = '69310'
fdff.loc[fdff.CODGEO_2024 == '75056', 'codepostal'] = '75000'
fdff.loc[fdff.CODGEO_2024 == '85165', 'codepostal'] = '85140'
fdff.loc[fdff.CODGEO_2024 == '85212', 'codepostal'] = '85140'

fdff.loc[fdff.CODGEO_2024 == '75056', ['area2015','pop2015','altitudemoy','gps']] = ['10534','2240621','35','48.866667, 2.333333']
fdff.loc[fdff.CODGEO_2024 == '13055', ['area2015','pop2015','altitudemoy','gps']] = ['23851','852516','179','43.296482, 5.36978']
fdff.loc[fdff.CODGEO_2024 == '69123', ['area2015','pop2015','altitudemoy','gps']] = ['4794','496343','237','45.764043, 4.835659']

In [ ]:
fdff = fdff.rename(columns={
        'topiaid': 'ex_topiaid',
        # -
        # 'topiaversion',
        'CODGEO_2024' : 'codeInsee',
        # 'timestamp',
        'LIBGEO_2024' : 'commune',
        'PRA_Code': 'petiteRegionAgricoleCode', 
        'PRA_Lib' : 'petiteRegionAgricoleNom',
        'dep_code' : 'departement',
        'codepostal' : 'codePostal',
        'reg_code' : 'region',
        # 'active',
        # 'gps',
        # 'refcountry',
        # +
        'altitudemoy' : 'altitudeMoyenne', 
        'area2015' : 'aire_ha_2015', 
        'pop2015' : 'pop_2015', 
        'aire_attr_2020': 'aireAttraction', 
        'arr_code' : 'arrondissementCode',
        'bassin_vie_2022': 'bassinVie',
        'intercomm_code' : 'intercommunaliteCode', 
        'unit_urb_2020' : 'uniteUrbaine', 
        'z_emploi_2020' : 'zoneEmploi',
        # 'GEOFLA2015_id',
        'Vineyard_basin' : 'bassinViticole', 
        'Region_old' : 'regionPre2016',
        'CODGEO_2022' : 'code_insee_2022',
        'CODGEO_2017' : 'code_insee_2017',
        'CODGEO_2014' : 'code_insee_2014',
        'CODGEO_2013' : 'code_insee_2013'
})

fdff[['latitude', 'longitude']] = fdff['gps'].str.split(', ', n=1, expand=True)
fdff['pays'] = 'fra'
fdff['active'] = 't'

In [ ]:
fdff = fdff[['codeInsee', 
             'commune', 
             'region', 
             'departement',
             'codePostal', 
            #  'topiaversion', 
            #  'timestamp',
             'petiteRegionAgricoleCode',
             'petiteRegionAgricoleNom',
             'pays',
             'latitude', 
             'longitude', 
             'altitudeMoyenne',
             'aireAttraction',
             'arrondissementCode',
             'bassinVie',
             'intercommunaliteCode',
             'uniteUrbaine',
             'zoneEmploi',
             'bassinViticole',
             'regionPre2016',
             'active',
            #  'refcountry', 
            #  'aire_ha_2015', 
            #  'pop_2015', 
             'GEOFLA2015_id',
            #  'code_insee_2022', 
            #  'code_insee_2017',
            #  'code_insee_2014', 
            #  'code_insee_2013',
             'temp_checking',
             'ex_topiaid']]

fdff.to_csv(path_csv + 'maj_reflocation_ticket13305.csv', index=False, sep=';')


In [ ]:
import sys
sys.exit('Attention deux parties de script. La suivante est faite pour créer le script de migration !')

In [1]:
# import scripts.interoperabilite as intop
import geopandas as gpd
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, URL
from getpass import getpass
from ast import literal_eval

In [ ]:
# psw = getpass("tbadie's password")
# # psw = psw.replace('@', '%40')

# url_agsdev_agsprodcopy = URL.create(
#     "postgresql+psycopg2",
#     username = "tbadie",
#     password = psw,
#     host = "147.100.179.208",
#     port = '5438',
#     database = "agrosyst_13167_20250130",
# )
# del(psw)

# connection = create_engine(url_agsdev_agsprodcopy)

In [2]:
#### ARRIVEE DE LA 3.6 

path = '/home/administrateur/Bureau/Datagrosyst/data_entrepot_&_outils/referentiels/geospatial_data/insee/'
df_refcomm = pd.read_csv(path + 'reflocation_3_6.csv', sep=';', low_memory=False)
df_refcomm = df_refcomm[['topiaid','codeinsee']]
# maj_other_ref = fdff.loc[(fdff['temp_checking']=='fusion') & (fdff['ex_topiaid'].str.contains('\[', na=True)), ['codeinsee','ex_topiaid']] 
maj_other_ref = pd.read_csv(path + 'maj_reflocation_ticket13305.csv', sep=';', low_memory=False)
# double précaution mais on devrait juste se baser sur temp_cheking
maj_other_ref = maj_other_ref.loc[(maj_other_ref['temp_checking']=='fusion') & (maj_other_ref['ex_topiaid'].str.contains('\[', na=True)), ['codeinsee','ex_topiaid']]

maj_other_ref['ex_topiaid'] = maj_other_ref['ex_topiaid'].apply(literal_eval)
maj_other_ref = maj_other_ref.explode('ex_topiaid')

ref = maj_other_ref.merge(df_refcomm, on = 'codeinsee', how='left')
ref = ref.loc[ref['ex_topiaid'] != ref['topiaid']]
# ref = ref[['ex_topiaid','topiaid']]

tables = ['practicedplot','plot','domain']

In [3]:
ref.to_csv(path + 'refcommune_fusionnee.csv', sep=';', index=False)

In [5]:
sql = []
for id, row in ref.iterrows() :
    for t in tables :
        sql.append('UPDATE public.'+ t +' SET location = \'' + str(row['topiaid']) + '\' WHERE location = \''+ str(row['ex_topiaid']) + '\' ;')
    sql.append('UPDATE public.reflocation SET active = false WHERE topiaid = \'' + row['ex_topiaid'] + '\' ;')

In [6]:
with open(path + "script_migration_reflocation.txt", "w") as output:
    output.write(str(sql))

# sql_refloc = pd.DataFrame({'reflocation_edit_active': [sql_refloc]})
# sql = pd.DataFrame({'other_table_edit_location_id': [sql]})
# sql_refloc.to_sql('reflocation_edit_active.sql', connection)
# sql.to_sql('other_table_edit_location_id.sql', connection)